In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from sklearn.metrics import mean_squared_error
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Mercari Kaggle Dataset
- What is Mercari?
    - Mercari is a website that allows people to sell/buy light used or new-ish products. Their slogan is literally "Sell or buy. Almost anything".
- What type of data is in this dataset
    - This dataset consist of item name, brand, description, category, condition and if shipping for the item is included or not. Most of the columns are pure text and need to be cleaned
- Purpose?
    - Predict the price of the items given the features mentioned above


# Methods

- stem_str_item_description: function to clean text in item description
- clean_str_brand_name: function to clean text in brand name
- impute_category_name: function to clean text in category name
- rmsle: calculates the root mean square log error
- word_freq: gets the count of unique words within a column

In [3]:
# Stems words to their root words and removes all characters that are not alphabets
def stem_str_item_description(str):
    ret_str = ""
    for w in word_tokenize(str.lower()):
        if w not in stop_words and w.isalpha() and len(w) > 1:
            ret_str = ret_str + " " + ps.stem(w)
    ret_str = re.sub("[^a-zA-Z]", " ", ret_str)
    return ret_str.strip()

In [4]:
# Stems words to their root words and removes all characters that are not alphabets
def clean_str_brand_name(str):
    ret_str = ""
    for w in str.lower():
        if w.isalnum() and len(w) > 0:
            ret_str = ret_str + w
#     ret_str = re.sub("[^a-zA-Z]", "", ret_str)
    return ret_str.strip()

In [5]:
def impute_category_name(row):
    if pd.isnull(row['category_name'])== True:
        if row['clean_brand_name'] in brand_cat_dict.keys():
            return brand_cat_dict[row['clean_brand_name']]
        else:
            return "No category name"
    else:
        return row['category_name']

In [6]:
# code copied from https://www.kaggle.com/marknagelberg/rmsle-function
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

In [7]:
# Gets the count of most frequent words give a dataframe
def word_freq(df, col):
    word_frequency = {}
    word_frequency_lst = []
    for index,row in df.iterrows(): 
        for w in list(set(row[col].split(' '))):
            if w not in word_frequency:
                word_frequency[w] = 1
            else:
                word_frequency[w] += 1

    for key, value in word_frequency.items():
        temp = [key, value]
        word_frequency_lst.append(temp)
    word_freq_df = pd.DataFrame(word_frequency_lst, columns=["unique_word", 'frequency'])
    word_freq_df = word_freq_df.sort_values(['frequency'], ascending=False)
    return word_freq_df

# Read Data

In [8]:
unclean_train_data = pd.read_table('/Users/joashc/Downloads/mercari-price-suggestion-challenge/train.tsv')
unclean_train_data.shape

/Users/joashc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


(1482535, 8)

In [10]:
unclean_train_data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


# Clean Data

### Removing Unwanted Columns and Duplicate Rows

In [9]:
unclean_train_data_v2 = unclean_train_data.drop(columns='train_id')

In [10]:
unclean_train_data_v3 = unclean_train_data_v2.drop_duplicates()
unclean_train_data_v3.shape

(1482486, 7)

### Dealing with Nulls

In [11]:
unclean_train_data_v3.isnull().sum()

name                      0
item_condition_id         0
category_name          6327
brand_name           632641
price                     0
shipping                  0
item_description          4
dtype: int64

#### Brand Name Nulls

In [12]:
brand_value_counts = (pd.DataFrame(unclean_train_data_v3.brand_name.value_counts())
    .reset_index()
    .rename(columns={'index':'brand',
                    'brand_name':'count_rows'}))

print('Number of brands:', brand_value_counts.shape[0]-1)
brand_value_counts.head()

Number of brands: 4808


,brand,count_rows
0,PINK,54088
1,Nike,54043
2,Victoria's Secret,48035
3,LuLaRoe,31024
4,Apple,17322


Fill brand name nulls with "nobrandname"

In [13]:
unclean_train_data_v3['brand_name'] = unclean_train_data_v3['brand_name'].fillna("nobrandname")

/Users/joashc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
unclean_train_data_v3.isnull().sum()

name                    0
item_condition_id       0
category_name        6327
brand_name              0
price                   0
shipping                0
item_description        4
dtype: int64

#### Item Description Nulls

In [15]:
print('Approx', unclean_train_data_v3.item_description.isnull().sum()/unclean_train_data_v3.shape[0], 
      '% of the dataset contains item_description that are null.')
print()
print('I am not deleting these rows because if the test data contains nulls in the item_description,',
     'I want to impute those values.')

Approx 2.698170505488753e-06 % of the dataset contains item_description that are null.

I am not deleting these rows because if the test data contains nulls in the item_description, I want to impute those values.


In [16]:
print('Unique item descriptions:', unclean_train_data_v3.item_description.nunique())

Unique item descriptions: 1281426


In [17]:
pd.DataFrame(unclean_train_data_v3.item_description.value_counts()).head()

,item_description
No description yet,82489
New,4099
Brand new,3058
Good condition,1274
Great condition,1158


Fill null item descriptions with "No description yet" as there are 82489 items that currently do not have any descriptions

In [18]:
unclean_train_data_v3['item_description'] = unclean_train_data_v3['item_description'].fillna('No description yet')

/Users/joashc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
unclean_train_data_v3.isnull().sum()

name                    0
item_condition_id       0
category_name        6327
brand_name              0
price                   0
shipping                0
item_description        0
dtype: int64

#### category_name nulls

In [20]:
# print('Approx', round(unclean_train_data_v3.category_name.isnull().sum()/unclean_train_data_v3.shape[0], 3), 
#       '% of the dataset contains category_name that are null.')
# print()
# print('I am not deleting these rows because if the test data contains nulls in the category_name,',
#      'I want to impute those values.')

Approx 0.004 % of the dataset contains category_name that are null.

I am not deleting these rows because if the test data contains nulls in the category_name, I want to impute those values.


In [21]:
# print("There are", 
#      unclean_train_data_v3[(unclean_train_data_v3['item_description']!='No description yet') & 
#                      (unclean_train_data_v3['brand_name']!='nobrandname') & 
#                           (unclean_train_data_v3['category_name'].isnull()==True)].shape[0],
#      "null values for category_name that have an item_description and brand_name")
# print()
# print("There are", 
#      unclean_train_data_v3[(unclean_train_data_v3['item_description']=='No description yet') & 
#                      (unclean_train_data_v3['brand_name']!='nobrandname') & 
#                           (unclean_train_data_v3['category_name'].isnull()==True)].shape[0],
#      "null values for category_name that have an brand_name but NO item_description")
# print()
# print("There are", 
#      unclean_train_data_v3[(unclean_train_data_v3['item_description']!='No description yet') & 
#                      (unclean_train_data_v3['brand_name']=='nobrandname') & 
#                           (unclean_train_data_v3['category_name'].isnull()== True)].shape[0],
#      "null values for category_name that have a item_description but NO brand_name")
# print()
# print("There are", 
#      unclean_train_data_v3[(unclean_train_data_v3['item_description']=='No description yet') & 
#                      (unclean_train_data_v3['brand_name']=='nobrandname') & 
#                           (unclean_train_data_v3['category_name'].isnull()== True)].shape[0],
#      "null values for category_name that have NO brand_name and NO item_description")

There are 2676 null values for category_name that have an item_description and brand_name

There are 194 null values for category_name that have an brand_name but NO item_description

There are 3029 null values for category_name that have a item_description but NO brand_name

There are 428 null values for category_name that have NO brand_name and NO item_description


In [22]:
# pd.DataFrame(unclean_train_data_v3.category_name.value_counts()).head()

,category_name
"Women/Athletic Apparel/Pants, Tights, Leggings",60176
Women/Tops & Blouses/T-Shirts,46380
Beauty/Makeup/Face,34331
Beauty/Makeup/Lips,29908
Electronics/Video Games & Consoles/Games,26557


In [23]:
# unclean_train_data_v3['category_name'] = np.where((unclean_train_data_v3['item_description']=='No description yet') &
#                                                   (unclean_train_data_v3['brand_name']=='nobrandname') &
#                                                    (unclean_train_data_v3['category_name'].isnull()== True),
#                                                   "No category",
#                                                   unclean_train_data_v3['category_name']
# )

# print(unclean_train_data_v3['category_name'].isnull().sum(), 'null values for category_name left!')

5899 null values for category_name left!


/Users/joashc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


For imputing the rest of the nulls for category_name, I want to use the item_description and brand_name columns. I first have to clean these columns to deal with the rest of the nulls.

### Initial Text Cleaning
- brand_name
    - Make everything lower case
    - Take out spaces
    - Take out special characters
- item_description
    - Make everything lower case
    - Take out special characters and numeric characters
    - Tokenize
    - Removal of stopwords

In [38]:
%%time
unclean_train_data_v3['stemmed_item_description'] = unclean_train_data_v3['item_description'].apply(
    lambda x: stem_str_item_description(str(x)))

CPU times: user 13min 29s, sys: 4.17 s, total: 13min 34s
Wall time: 13min 57s


/Users/joashc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
# replace nulls after cleaning with default talenized value
unclean_train_data_v3['stemmed_item_description'] = np.where(
    unclean_train_data_v3['stemmed_item_description'].isnull()==True,
'descript yet', unclean_train_data_v3['stemmed_item_description'])

In [14]:
%%time
unclean_train_data_v3['clean_brand_name'] = unclean_train_data_v3['brand_name'].apply(
    lambda x: clean_str_brand_name(str(x)))

CPU times: user 2.82 s, sys: 47.8 ms, total: 2.87 s
Wall time: 2.88 s


In [54]:
unclean_train_data_v3.isnull().sum()

name                           0
item_condition_id              0
category_name               5899
brand_name                     0
price                          0
shipping                       0
item_description               0
stemmed_item_description    3320
clean_brand_name               0
dtype: int64

### Dealing with category_name Nulls

In [55]:
# unique_brand_names_w_na_category = (list(unclean_train_data_v3[
#     (unclean_train_data_v3['category_name'].isnull()== True)].clean_brand_name.unique()))

# print(len(unique_brand_names_w_na_category), 'unique brand names with null category names')

606 unique brand names with null category names


In [56]:
# %%time
# brand_cat_dict = {}
# for brand in unique_brand_names_w_na_category:
#     subset_brand_data = unclean_train_data_v3[(unclean_train_data_v3['clean_brand_name']==brand) & 
#                                              (unclean_train_data_v3['category_name'].isnull()==False)]
#     if subset_brand_data.shape[0] > 1:
#         # can only perform value counts of shape of dataset is greater than 1
#         brand_cat_dict[brand] = pd.DataFrame(subset_brand_data.category_name.value_counts()).head(1).index[0]
# print('Found most popular categories for', len(brand_cat_dict), 'brand names.',
#      'For the rest of the brand names where we could not find the most popular category, we will,'
#      ' impute no categpry for now.')

# for brand in list(set(unique_brand_names_w_na_category) - set(brand_cat_dict.keys())):
#     brand_cat_dict[brand] = "No category"

Found most popular categories for 579 brand names. For the rest of the brand names where we could not find the most popular category, we will, impute no categpry for now.
CPU times: user 51 s, sys: 144 ms, total: 51.1 s
Wall time: 51.3 s


In [33]:
# test_df = unclean_train_data_v3[unclean_train_data_v3['category_name'].isnull()==True]

In [58]:
# unclean_train_data_v3['category_name'] = unclean_train_data_v3.apply(impute_category_name, axis=1)

In [63]:
# unclean_train_data_v3.isnull().sum()

name                        0
item_condition_id           0
category_name               0
brand_name                  0
price                       0
shipping                    0
item_description            0
stemmed_item_description    0
clean_brand_name            0
dtype: int64

### Clean category_name Column

In [67]:
# replace / with a space
unclean_train_data_v3['category_name'] = unclean_train_data_v3['category_name'].str.replace('/',' ')

In [71]:
%%time
unclean_train_data_v3['clean_category_name'] = unclean_train_data_v3['category_name'].apply(
    lambda x: stem_str_item_description(str(x)))

CPU times: user 3min 48s, sys: 1.28 s, total: 3min 49s
Wall time: 3min 52s


In [74]:
unclean_train_data_v3['clean_category_name'].isnull().sum()

0

In [66]:
unclean_train_data_v3['clean_category_name'] = unclean_train_data_v3['clean_category_name'].fillna('unknown')

### Clean name column

In [52]:
# replace / with a space
unclean_train_data_v3['name'] = unclean_train_data_v3['name'].str.replace('/',' ')

In [53]:
%%time
unclean_train_data_v3['clean_item_name'] = unclean_train_data_v3['name'].apply(
    lambda x: stem_str_item_description(str(x)))

CPU times: user 3min 39s, sys: 1.17 s, total: 3min 40s
Wall time: 3min 42s


In [68]:
unclean_train_data_v3['clean_item_name'].isnull().sum()

0

In [67]:
unclean_train_data_v3['clean_item_name'] = unclean_train_data_v3['clean_item_name'].fillna('unknown')

In [62]:
# unclean_train_data_v3[unclean_train_data_v3['clean_item_name'].isnull()==True].name.value_counts()

### Assigning Catgeories and Sub Categories based on Mercari site

In [9]:
mercari_category = pd.read_csv('~/mercari_price_suggestion/mercari_price_suggestion/mercari_category.csv')
print(mercari_category.shape)
mercari_category.head(1)

(161, 2)


,category,sub_category
0,Women,Dresses


In [4]:
# %%time
# mercari_category['stemmed_cat'] = mercari_category['category'].apply(
#     lambda x: stem_str_item_description(str(x)))

#### Assigning Categories

In [21]:
mercari_category_lst = ['Women', 'Toys', 'Vintage', 'Electronics', 'Home', 'Beauty',
      'Handmade', 'Sports',  'Men', 'Kids', 'Other']

In [ ]:
# Electronics: ['TV', 'Tablets & E-readers', 'Vinatage']
#     put electronics after vintage
# Home: ['Home décor', 'Other']
# Toys: ['Learning & Education', 'Games & Puzzles', 'Dress Up & Pretend Play', 'Electronics for Kids']
# Women: ['Shoes', "Women's accessories", 'Athletic apparel', 'Other']
# vintage: ['Toy', 'Other']
# Men: simply move down the list
# Sports: put sports after Vintage and handmade
# Kids: Put it last

In [22]:
def assign_category(row, cat_lst=mercari_category_lst):
    for cat in cat_lst:
        if row['category_name'].lower().find(cat.lower()) >=0:
            return cat

In [11]:
# test = unclean_train_data_v3.sample(100)

In [23]:
unclean_train_data_v3['assigned_category'] = unclean_train_data_v3.apply(assign_category, axis=1)
unclean_train_data_v3.shape

(1482486, 13)

In [24]:
unclean_train_data_v3.assigned_category.isnull().sum()

457

In [25]:
unclean_train_data_v3['assigned_category'] = unclean_train_data_v3['assigned_category'].fillna('Other')

#### Assigning Sub Categories

In [26]:
sub_category_dict = {}
for cat in mercari_category.category.unique():
    sub_category_dict[cat] = []

for index, row in mercari_category.iterrows():
    sub_category_dict[row['category']].append(row['sub_category'])    

In [27]:
def assign_sub_category(row, sub_cat_dict=sub_category_dict):
    sub_cat_lst = sub_category_dict[row['assigned_category']]
    for sub_cat in sub_cat_lst:
        if row['category_name'].lower().find(sub_cat.lower()) >=0:
            return sub_cat

In [28]:
%%time
unclean_train_data_v3['assigned_sub_category'] = unclean_train_data_v3.apply(assign_sub_category, axis=1)
print(unclean_train_data_v3.shape)

(1482486, 13)
CPU times: user 1min 27s, sys: 713 ms, total: 1min 28s
Wall time: 1min 29s


In [29]:
unclean_train_data_v3.assigned_sub_category.isnull().sum()

103216

Could work on better resolving the categories. But they are good enough for now. Will impute the nulls with "Other"

In [47]:
unclean_train_data_v3['assigned_sub_category'] = unclean_train_data_v3['assigned_sub_category'].fillna('Other')

In [51]:
unclean_train_data_v3.shape

(1482486, 13)

In [43]:
# test = unclean_train_data_v3[(unclean_train_data_v3['assigned_sub_category'].isnull()==True) & 
#                             (unclean_train_data_v3['assigned_category']=='Toys')]

In [48]:
# test[['category_name', 'assigned_category', 'assigned_sub_category']].drop_duplicates()

In [49]:
# test.assigned_category.value_counts()

In [69]:
# unclean_train_data_v3.to_csv(
#     '/Users/joashc/Downloads/mercari-price-suggestion-challenge/partially_clean_train_data.csv', 
#     index=False)

In [57]:
unclean_train_data_v3.isnull().sum()

name                           0
item_condition_id              0
category_name                  0
brand_name                     0
price                          0
shipping                       0
item_description               0
stemmed_item_description       0
clean_brand_name               0
clean_category_name         7932
clean_item_name             4246
assigned_category              0
assigned_sub_category          0
dtype: int64

In [64]:
unclean_train_data_v3 = pd.read_csv(
    '/Users/joashc/Downloads/mercari-price-suggestion-challenge/partially_clean_train_data.csv')
unclean_train_data_v3.shape

(1482486, 13)

## item_condition

In [54]:
item_condition = pd.concat([pd.DataFrame(unclean_train_data_v3.item_condition_id.value_counts()),
           pd.DataFrame(unclean_train_data_v3.item_condition_id.value_counts(normalize=True))], 
          axis=1)

item_condition.columns = ['item_condition_count', 'item_condition_percent']
item_condition.head()

,item_condition_count,item_condition_percent
1,640501,0.432045
3,432161,0.291511
2,375478,0.253276
4,31962,0.021560
5,2384,0.001608


In [59]:
item_condition_agg = (unclean_train_data_v3
    .groupby('item_condition_id')
    .agg({'price':['min', 'median', 'mean', 'max']})
    .reset_index()
)

item_condition_agg

item_condition_id price                          
                      min median       mean     max
0                 1   0.0   18.0  26.488162  2009.0
1                 2   0.0   17.0  27.563231  2004.0
2                 3   0.0   16.0  26.540711  2000.0
3                 4   0.0   15.0  24.349212  1309.0
4                 5   0.0   19.0  31.703859   522.0

This aggregation of item condition and the price does not really tell us much as the dataset contains difference categories of items like clothing, jewelery, etc

# Future Improvements

- Replace / with a space
- Get stuff out of the description of category (size, oz, etc)
- standardizing the categories (Clothes, Jewelery, Other)
- get gender for each item (men, women, child, both, other(animal))
- assign categories and sub categories null values better
- take out special characters and -, / before tokenizing
- Brand name from item name
- Sport category didnt get the sub categories well
